In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset

# -*- coding: utf-8 -*-

# import the normalized slices
slices = pd.read_csv('data/slices_normalized.csv', index_col=0)

# Convert the DataFrame to a numpy array, then to a PyTorch Tensor
slices_tensor = torch.tensor(slices.values)

# Convert your data to a TensorDataset
slices_dataset = TensorDataset(slices_tensor)

# DEFINES THE BATCH SIZE
batch_size = 32

# Create a DataLoader with shuffle=True for shuffling at each epoch
slices_loader = DataLoader(slices_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

/var/folders/b4/spskmk5x6xggprwkxqclxbb40000gn/T/ipykernel_65883/4277157898.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# import the backward file and functions
import backward
from torch.optim import Adam

# Create an instance of the denoising network
model = backward.DenoisingNetwork()


device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = Adam(model.parameters(), lr=0.00001)
epochs = 100
T = backward.T

# Train the model
print("Training the model...")
for epoch in tqdm(range(epochs)):
    for step, batch in enumerate(slices_loader):
        optimizer.zero_grad()

        t = torch.randint(0,T, (batch_size,), device=device).long()
        
        loss = backward.get_loss(model, batch[0], t) 
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0 and step == 0:
            print(f"Epoch {epoch} | step {step:03d}, Loss {loss.item()}")
            backward.sample_plot_TS(model)        

/Users/thedog/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Training the model...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0 | step 000, Loss 0.8677729020163159


  0%|          | 0/100 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def sample_TS(model):
    x = torch.randn(1, 100)
    for i in range(0, T)[::-1]:
        t = torch.full((1,), i ,device = device, dtype=torch.long)
        x = backward.sample_timestep(x, t, model)
    return x

# use the model to generate 1000 samples
samples = []
for i in tqdm(range(1000)):
    TS = sample_TS(model)
    # Normalize the tensor between -1 and 1
    TS_norm = 2 * (TS - TS.min()) / (TS.max() - TS.min()) - 1
    # Convert tensor to numpy array and flatten it to 1D
    samples.append(TS_norm.squeeze(0).cpu().numpy())

# Convert the samples to a DataFrame
samples_df = pd.DataFrame(samples)

# Save the samples to a CSV file without index
samples_df.to_csv('data/generated_samples.csv', index=False)

100%|██████████| 1000/1000 [03:50<00:00,  4.35it/s]
